# P3Delta Online
<u>Version</u> : 0.4 (09/06/2021)

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code of P3Delta Online is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
# -----------------------------------------------------------------------------------------------------------------
# Package
# -----------------------------------------------------------------------------------------------------------------

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))
%matplotlib notebook
import numpy as np
import sympy as sp
import cxroots as cx
import scipy.special as spsp
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.cm as cm
import ipywidgets as widgets
from IPython.display import display
from IPython.display import update_display
from IPython.display import Code
from IPython.display import Markdown


# -----------------------------------------------------------------------------------------------------------------
# Computation function for back-end
# -----------------------------------------------------------------------------------------------------------------

def compute_root_spectrum(n,m,s0,value_tau):
    
    IntProgress_mid_generic.value = 3
    s = sp.symbols('s')  # define variable s for our problem to be solved
    tau = sp.symbols('tau')  # define variable tau : delay
    a = sp.symbols(["a{:d}".format(i) for i in range(n)], real=True)
    alpha = sp.symbols(["alpha{:d}".format(i) for i in range(m + 1)], real=True)
    Polynomial = s**n + np.array(a).dot([s**i for i in range(n)]) # Revient à faire s^n + a_{n-1}^{n-1}...
    Delayed = np.array(alpha).dot([s**i for i in range(m+1)])*sp.exp(-s*tau) # Revient à faire 
    Q = Polynomial + Delayed 
    SysDerivatif = [Q]
    IntProgress_mid_generic.value = 4
    for i in range(n+m+1):
        DerniereDerivee = SysDerivatif[-1]
        SysDerivatif.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s
    IntProgress_mid_generic.value = 5
    sol = sp.linsolve(SysDerivatif[:-1], alpha + a).args[0] # Solveur selon les alpha et les a
    solNum = sol.subs({s : s0})
    solNum = solNum.subs({tau : value_tau})
    a_num = list(solNum[m + 1:])
    alpha_num = list(solNum[:m + 1])
    QNumerique = s**n + np.array(a_num).dot([s**i for i in range(n)])+\
    np.array(alpha_num).dot([s**i for i in range(m+1)])*sp.exp(-s*tau)
    IntProgress_mid_generic.value = 6
    QNumerique = QNumerique.subs(tau, value_tau)
    sysRootFinding = [QNumerique, QNumerique.diff(s)]
    sysFunc = [sp.lambdify(s, i) for i in sysRootFinding]
    IntProgress_mid_generic.value = 7
    rect = cx.Rectangle([-100, 10], [-100, 100])
    roots = rect.roots(sysFunc[0], sysFunc[1], rootErrTol=1e-5, absTol=1e-5, M = n + m + 1)
    xroot = np.real(roots[0])
    yroot = np.imag(roots[0])
    IntProgress_mid_generic.value = 8
    return xroot,yroot,QNumerique

# -----------------------------------------------------------------------------------------------------------------

def compute_admissibilite(n,m,a_coeff_mid_co):
    
    s = sp.symbols('s')  # define variable s for our problem to be solved
    tau = sp.symbols('tau')  # define variable tau : delay

    a = sp.symbols(["a{:d}".format(i) for i in range(n)], real=True)
    alpha = sp.symbols(["alpha{:d}".format(i) for i in range(m + 1)], real=True)
    avalue = a_coeff_mid_co

    Polynomial = s**n + np.array(a).dot([s**i for i in range(n)]) # Revient à faire s^n + a_{n-1}^{n-1}...
    Delayed = np.array(alpha).dot([s**i for i in range(m+1)])*sp.exp(-s*tau) # Revient à faire 
    #b^m*s^m + b_{m-1}^{m-1}...
    Q = Polynomial + Delayed 

    SysDerivatif = [Q]
    for i in range(m+1):
        DerniereDerivee = SysDerivatif[-1]
        SysDerivatif.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s

    sol = sp.linsolve(SysDerivatif[:-1], alpha).args[0] # Solveur selon les alpha et les a

    polyAdm = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)})
    polyAdm = polyAdm.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    polyAdm = sp.simplify(polyAdm)

    s0range = np.arange(-10, 0, 0.01)
    taurange = np.arange(0, 10, 0.01)

    func = sp.lambdify([s, tau], polyAdm)

    return s0range,taurange,polyAdm,s,tau,Q,avalue,SysDerivatif,alpha,a,sol

def getRoots(m, Q, dominancy, delay, avalue, alphavalue, xwindow, ywindow,s,a,alpha,tau):
    derivees = [Q, Q.diff(s)]
    for i in range(len(derivees)) :
        derivees[i] = derivees[i].subs({ai: ai_num for ai, ai_num in zip(a, avalue)})
        derivees[i] = derivees[i].subs({alphai: alphai_num for alphai, alphai_num in zip(alpha, alphavalue)})
        derivees[i] = derivees[i].subs({tau : delay})
    func = [sp.lambdify(s, i) for i in derivees]
    rect = cx.Rectangle(xwindow, ywindow)
    root_count = rect.count_roots(func[0])
    roots = rect.roots(func[0], func[1], rootErrTol=1e-5, absTol=1e-5, M=m+2)
    xroot, yroot = np.real(roots[0]), np.imag(roots[0])
    return xroot, yroot

def solve_tau_connu(tau_val, acoef, Q,m,s,SysDerivatif,alpha,a,tau):
    sys = [Q]
    for i in range(m+1):
        DerniereDerivee = sys[-1]
        sys.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s
    sol = sp.linsolve(SysDerivatif[:-1], alpha).args[0]
    finaleq = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)}) 
    finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, acoef)})
    solS0 = finaleq.subs({tau : tau_val})
    solS0 = sp.solve(solS0)
    solS0eval = [i.evalf() for i in solS0]
    try :
        solution = max([i for i in solS0eval if i<0])
    except Exception :
        traceback.print_exc()
    return solution

def compute_sensibilite(value_tau,Q,m,s,SysDerivatif,alpha,a,tau):
    tau_nominal = value_tau
    step = 1e-2
    nbIt = 10
    values = [0] + [-step * i for i in range(1, nbIt + 1)] + [step * i for i in range(1, nbIt + 1)]
    values.sort()
    tau_sens = []
    s0_sens = []
    for value in values :
        tau_sens.append(tau_nominal + value)
        s0_sens.append(solve_tau_connu(tau_sens[-1], [2, 1], Q,m,s,SysDerivatif,alpha,a,tau))
        
    sensIterations = len(tau_sens)
    bList = np.linspace(start=255, stop=0, num=sensIterations//2) #Blue to black
    rList = np.linspace(start=0, stop=255, num=sensIterations//2) #black to red
    if len(bList)+len(rList)==sensIterations-1 :
        zerosToFill = [0] * (sensIterations//2 + 1)
    else :
        zerosToFill = [0] * (sensIterations//2 )
            
    normaliser = clr.Normalize(min(tau_sens), max(tau_sens))
    colormapper = cm.ScalarMappable(norm = normaliser , cmap = plt.get_cmap ("coolwarm") )
    colormapper.set_array(tau_sens)

    return tau_sens,s0_sens,colormapper

def factorization_integral_latex(n, m, s0, tau):
    factor = str(tau**(m+1) / spsp.factorial(m))
    parenthesis = "(s + " + str(-s0) + ")"
    power = "^{" + str(n + m + 1) + "}"
    
    return r"\$" + "\Delta(s) = " + factor + parenthesis + power + r"\int_0^1 t^{" + str(m) + r"} (1 - t)^{" + str(n) + "} e^{-" + str(tau) + "t" + parenthesis + "} \mathrm{d}t" + "$"

def factorization_1f1_latex(n, m, s0, tau):
    factor = str(tau**(m+1) * spsp.factorial(n) / spsp.factorial(n+m+1))
    parenthesis = "(s + " + str(-s0) + ")"
    power = "^{" + str(n + m + 1) + "}"
    
    return r"\$" + "\Delta(s) = " + factor + parenthesis + power + r" {}_1 F_1(" + str(m+1) + r", " + str(n+m+2) + ", -" + str(tau) + parenthesis + ")" + "$" 

# -----------------------------------------------------------------------------------------------------------------

def compute_CRRID_root (n,m,s0,d,value_tau):
    sList = [s0-i*d for i in range(n+m+1)]
    
    s = sp.symbols('s')  # define variable s for our problem to be solved
    tau = sp.symbols('tau')  # define variable tau : delay

    a = sp.symbols(["a{:d}".format(i) for i in range(n)], real=True)
    alpha = sp.symbols(["alpha{:d}".format(i) for i in range(m + 1)], real=True)
    
    
    Polynomial = s**n + np.array(a).dot([s**i for i in range(n)]) # Revient à faire s^n + a_{n-1}^{n-1}...
    Delayed = np.array(alpha).dot([s**i for i in range(m+1)])*sp.exp(-s*tau) # Revient à faire 
    #b^m*s^m + b_{m-1}^{m-1}...
    Q = Polynomial + Delayed 
    
    Sys = [Q] * (n + m + 1)
    for i in range(n + m + 1):
        Sys[i] = Sys[i].subs({s: sList[i]})
        Sys[i] = Sys[i].subs({tau: value_tau})
        
    sol = sp.linsolve(Sys, alpha + a).args[0] # Solveur selon les alpha et les a
    
    solNum = sol.subs({tau : value_tau})
    a_num = list(solNum[m + 1:])
    alpha_num = list(solNum[:m + 1])
    
    QNumerique = s**n + np.array(a_num).dot([s**i for i in range(n)])+\
    np.array(alpha_num).dot([s**i for i in range(m+1)])*sp.exp(-s*tau)
    QNumerique = QNumerique.subs(tau, value_tau)
    
    sysRootFinding = [QNumerique, QNumerique.diff(s)]
    sysFunc = [sp.lambdify(s, i) for i in sysRootFinding]
    rect = cx.Rectangle([-100, 10], [-100, 100])
    roots = rect.roots(sysFunc[0], sysFunc[1], rootErrTol=1e-5, absTol=1e-5, M = n + m + 1)
    xroot = np.real(roots[0])
    yroot = np.imag(roots[0])
    
    return xroot,yroot,QNumerique
    

# -----------------------------------------------------------------------------------------------------------------
# Output
# -----------------------------------------------------------------------------------------------------------------

output_root_spectrum_computation_mid_generic = widgets.Output()
output_output_equation_computation_mid_generic = widgets.Output()
output_factorized_integral_equation_computation_mid_generic = widgets.Output()
output_factorization_1f1_equation_computation_mid_generic = widgets.Output()

# -----------------------------------------------------------------------------------------------------------------

output_admissibilite_computation_mid_co = widgets.Output()
output_solution_mid_co = widgets.Output()
output_output_equation_computation_mid_co = widgets.Output()
output_sensibilite_computation_mid_co = widgets.Output()

# -----------------------------------------------------------------------------------------------------------------

output_CRRID_root_computation = widgets.Output()
output_output_equation_computation_CRRID = widgets.Output()

# -----------------------------------------------------------------------------------------------------------------
# User Interface input
# -----------------------------------------------------------------------------------------------------------------

slider_n_mid_generic = widgets.IntSlider(min=0,max=7,step=1,description='n :',value=2)

def slider_n_mid_generic_eventhandler(change):
    
    slider_m_mid_generic.max = change.new
    Text_error_mid_generic.value = "Be aware ! n should be higher or equal to m ! "
        
slider_n_mid_generic.observe(slider_n_mid_generic_eventhandler, names='value')

slider_m_mid_generic = widgets.IntSlider(min=0,max=7,step=1,description='m :',value=1)

FloatText_s0_mid_generic = widgets.BoundedFloatText(min=-10.00,max=0.00,step=0.01,description='s0 :',value=-5.00,disabled=False)
FloatText_tau_mid_generic = widgets.BoundedFloatText(min=0.50,max=2.00,step=0.01,description='tau :',value=1.00,disabled=False)

layout_button_ready_to_compute_mid_generic = widgets.Layout( width='100%')
layout_IntProgress_mid_generic = widgets.Layout( width='75%')
layout_valid_statut_mid_generic = widgets.Layout( width='99%') #, border='2px solid #8e2042'

button_ready_to_compute_mid_generic = widgets.Button(description='Compute',layout = layout_button_ready_to_compute_mid_generic, disabled=False,button_style='success',icon='check-circle') # style={'button_color': '#bb2a58','font_weight': 'bold'}
IntProgress_mid_generic = widgets.IntProgress(layout = layout_IntProgress_mid_generic,  value=0,min=0,max=10,description='Loading:',bar_style='success',orientation='horizontal') # style={'bar_color': '#8e2042'}, 'success', 'info', 'warning', 'danger' or ''
valid_statut_mid_generic = widgets.Valid(value=False,description='Status :',layout = layout_valid_statut_mid_generic)


layout_validate_mid_generic = widgets.AppLayout(header=None,left_sidebar=button_ready_to_compute_mid_generic,center=IntProgress_mid_generic,right_sidebar=valid_statut_mid_generic,footer=None)

Text_error_mid_generic = widgets.Text(value='It is all good',placeholder='Information may appear here',description='',disabled=True,layout = widgets.Layout( width='auto') )
# -----------------------------------------------------------------------------------------------------------------

slider_n_mid_co = widgets.IntSlider(min=0,max=7,step=1,description='n :',value=2)

def slider_n_mid_co_eventhandler(change):
    
    slider_m_mid_co.max = change.new
        
slider_n_mid_co.observe(slider_n_mid_co_eventhandler, names='value')

slider_m_mid_co = widgets.IntSlider(min=0,max=7,step=1,description='m :',value=1)

layout_button_confirm_degrees_mid_co = widgets.Layout( width='18%')
button_confirm_degrees_mid_co = widgets.Button(description='Confirm degrees',layout = layout_button_confirm_degrees_mid_co, disabled=False,button_style='info',icon='check-circle')

l = 9
items_IntText_a_mid_co = [widgets.BoundedIntText(min=-10,max=10,step=1,description='',value=0,disabled=True) for i in range(l-1)]
layout_IntText_a_mid_co = widgets.Layout(width='auto',max_width='81%',height='')

sub_list_1 = []
sub_list_2 = []

for k in range (0,4) :
    sub_list_1.append(items_IntText_a_mid_co[k])

for k in range (4,len(items_IntText_a_mid_co)) :
    sub_list_2.append(items_IntText_a_mid_co[k])

IntText_a_1_mid_co = widgets.Box(children=sub_list_1, layout=layout_IntText_a_mid_co)
IntText_a_2_mid_co = widgets.Box(children=sub_list_2, layout=layout_IntText_a_mid_co)

layout_button_confirm_system_coeff_mid_co = widgets.Layout( width='16%')
button_confirm_system_coeff_mid_co = widgets.Button(description='Confirm coefficients',layout = layout_button_confirm_system_coeff_mid_co, disabled=False,button_style='info',icon='check-circle')

ToggleButtons_mid_co = widgets.ToggleButtons(options=['tau', 's0'], description ='parameter',disabled=False,button_style='',tooltips=['tau is known', 's0 is known'])
ToggleButtons_mid_co.style.button_width='auto'

def ToggleButtons_mid_co_eventhandler(change):
    
    if change.new == 's0' :
        FloatText_s0_tau_mid_co.min,FloatText_s0_tau_mid_co.max = -10.000,0.000
        FloatText_s0_tau_mid_co.step = 0.001
        FloatText_s0_tau_mid_co.value = -1.506
        FloatText_s0_tau_mid_co.description = 's0 : '
    
    if change.new == 'tau' :
        FloatText_s0_tau_mid_co.max,FloatText_s0_tau_mid_co.min = 2.00,0.50
        FloatText_s0_tau_mid_co.step = 0.01
        FloatText_s0_tau_mid_co.value = 0.73
        FloatText_s0_tau_mid_co.description = 'tau : '


ToggleButtons_mid_co.observe(ToggleButtons_mid_co_eventhandler, names='value')

FloatText_s0_tau_mid_co = widgets.BoundedFloatText(min=0.50,max=2.00,step=0.01,description='tau : ',value=0.73,disabled=False)
                                     
layout_button_ready_to_compute_mid_co = widgets.Layout( width='100%')
layout_IntProgress_mid_co = widgets.Layout( width='75%')
layout_valid_statut_mid_co = widgets.Layout( width='99%') #, border='2px solid #8e2042'

button_ready_to_compute_mid_co = widgets.Button(description='Compute',layout = layout_button_ready_to_compute_mid_co, disabled=False,button_style='success',icon='check-circle')
IntProgress_mid_co = widgets.IntProgress(layout = layout_IntProgress_mid_co,  value=0,min=0,max=10,description='Loading:',bar_style='success',orientation='horizontal') # style={'bar_color': '#8e2042'}, 'success', 'info', 'warning', 'danger' or ''
valid_statut_mid_co = widgets.Valid(value=False,description='Status :',layout = layout_valid_statut_mid_co)

layout_validate_mid_co = widgets.AppLayout(header=None,left_sidebar=button_ready_to_compute_mid_co, center=IntProgress_mid_co, right_sidebar=valid_statut_mid_co,footer=None)

Text_error_mid_co = widgets.Text(value='It is all good',placeholder='Information may appear here',description='',disabled=True,layout = widgets.Layout( width='auto') )

# -----------------------------------------------------------------------------------------------------------------

slider_n_CRRID = widgets.IntSlider(min=0,max=7,step=1,description='n :',value=2)

def slider_n_CRRID_eventhandler(change):
    
    slider_m_CRRID.max = change.new
        
slider_n_CRRID.observe(slider_n_CRRID_eventhandler, names='value')

slider_m_CRRID = widgets.IntSlider(min=0,max=7,step=1,description='m :',value=1)
slider_d_CRRID = widgets.IntSlider(min=0,max=5,step=1,description='d :',value=2)

FloatText_s0_CRRID = widgets.BoundedFloatText(min=-10.00,max=0.00,step=0.01,description='s0 :',value=-1.00,disabled=False)
FloatText_tau_CRRID = widgets.BoundedFloatText(min=0.50,max=2.00,step=0.01,description='tau :',value=1.00,disabled=False)

layout_button_ready_to_compute_CRRID = widgets.Layout( width='100%')
layout_IntProgress_CRRID = widgets.Layout( width='75%')
layout_valid_statut_CRRID = widgets.Layout( width='99%') #, border='2px solid #8e2042'

button_ready_to_compute_CRRID = widgets.Button(description='Compute',layout = layout_button_ready_to_compute_CRRID, disabled=False,button_style='success',icon='check-circle')
IntProgress_CRRID = widgets.IntProgress(layout = layout_IntProgress_CRRID,  value=0,min=0,max=10,description='Loading:',bar_style='success',orientation='horizontal') # style={'bar_color': '#8e2042'}, 'success', 'info', 'warning', 'danger' or ''
valid_statut_CRRID = widgets.Valid(value=False,description='Status :',layout = layout_valid_statut_CRRID)

layout_validate_CRRID_0 = widgets.AppLayout(header=None,left_sidebar=FloatText_s0_CRRID,center=None,right_sidebar=FloatText_tau_CRRID,footer=None)
layout_validate_CRRID_1 = widgets.AppLayout(header=None,left_sidebar=button_ready_to_compute_CRRID,center=IntProgress_CRRID,right_sidebar=valid_statut_CRRID,footer=None)

Text_error_CRRID = widgets.Text(value='It is all good',placeholder='Information may appear here',description='',disabled=True,layout = widgets.Layout( width='auto') )

# -----------------------------------------------------------------------------------------------------------------
# Computation function for front-end
# -----------------------------------------------------------------------------------------------------------------

def computation_mid_generic(n,m,s0,value_tau):
    
    output_root_spectrum_computation_mid_generic.clear_output()
    output_output_equation_computation_mid_generic.clear_output()
    output_factorized_integral_equation_computation_mid_generic.clear_output()
    output_factorization_1f1_equation_computation_mid_generic.clear_output()
    
    with output_root_spectrum_computation_mid_generic:
        
        fig, ax = plt.subplots(figsize=(9, 5))
        ax.set_title("Spectrum distribution")
        ax.set_xlabel(r"$\Re(s)$")
        ax.set_ylabel(r"$\Im(s)$")
        ax.axhline(linewidth=2, color='black', zorder = 2)
        ax.axvline(linewidth=2, color='black', zorder = 2)
        ax.axvspan(0, 100, alpha=0.5, color='red', zorder = 2, label='Unstable system')
        ax.grid()
        line, = ax.plot([], [], 'o', color='steelblue', zorder = 3, label='Roots')
        line2, = ax.plot([], [], '*',markersize=20, color='#ff9900', zorder = 3,label='Rightmost root')
        
        xr,yr,eq = compute_root_spectrum(n,m,s0,value_tau)
        xmax = max(xr)
        plt.xlim([-10,1])
        line.set_data(xr, yr)
        line2.set_data(xmax,0)
        ax.legend(loc='upper right')
        ax.relim()
        #ax.autoscale_view()
        fig.canvas.draw()
        IntProgress_mid_generic.value = 9
    
    with output_output_equation_computation_mid_generic:
        print ("Fp = ",eq)
    
    with output_factorized_integral_equation_computation_mid_generic:
        factorization_integral_latex_equation_mid_generic = factorization_integral_latex(n, m, s0, value_tau)
        display(Markdown(factorization_integral_latex_equation_mid_generic))
    
    with output_factorization_1f1_equation_computation_mid_generic:
        factorization_1f1_latex_equation_mid_generic = factorization_1f1_latex(n, m, s0, value_tau)
        display(Markdown(factorization_1f1_latex_equation_mid_generic))
        IntProgress_mid_generic.value = 10

def button_ready_to_compute_clicked_mid_generic(self):
    
    valid_statut_mid_generic.value = False
    IntProgress_mid_generic.value = 1
    
    n_mid_generic = slider_n_mid_generic.value
    m_mid_generic = slider_m_mid_generic.value
    s0_mid_generic = FloatText_s0_mid_generic.value
    tau_mid_generic = FloatText_tau_mid_generic.value
    
    IntProgress_mid_generic.value = 2
    
    computation_mid_generic(n_mid_generic,m_mid_generic,s0_mid_generic,tau_mid_generic)
    
    IntProgress_mid_generic.value = 10
    valid_statut_mid_generic.value = True
    
button_ready_to_compute_mid_generic.on_click(button_ready_to_compute_clicked_mid_generic)

# -----------------------------------------------------------------------------------------------------------------

def computation_1_mid_co(n,m,a_coeff_mid_co):
        
    output_admissibilite_computation_mid_co.clear_output()
    
    with output_admissibilite_computation_mid_co:
        
        # additional calculations for the plot 
        
        s0range,taurange,polyAdm,s,tau,Q,avalue,SysDerivatif,alpha,a,sol = compute_admissibilite(n,m,a_coeff_mid_co)
        
        s0range = np.arange(-16, 0, 0.01)
        taurange = np.arange(0, 10, 0.01)

        func = sp.lambdify([s, tau], polyAdm)
        
        fig, ax = plt.subplots(figsize=(9, 5))
        X, Y = np.meshgrid(s0range, taurange)
        z = func(X, Y)
        CS = ax.contour(X, Y, z, [0])
        ax.grid()
        plt.xlabel(r"$s_0$",fontsize = 12)
        plt.ylabel(r"tau ($\tau$)")
        plt.title("Admissibility plot")
        
        

def computation_2_mid_co(n,m,a_coeff_mid_co,user_choice,user_value):
    
    output_solution_mid_co.clear_output()
    output_output_equation_computation_mid_co.clear_output()
    output_sensibilite_computation_mid_co.clear_output()
    
    with output_solution_mid_co :
        
        s0range,taurange,polyAdm,s,tau,Q,avalue,SysDerivatif,alpha,a,sol = compute_admissibilite(n,m,a_coeff_mid_co)
        
        finaleq = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)}) #remplace les coeffs
        finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, avalue)})
        
        if user_choice == 'tau' :
            
            value_tau = user_value
            
            solS0 = finaleq.subs({tau : value_tau})
            solS0 = sp.solve(solS0)
            solS0eval = [i.evalf() for i in solS0]
            
            computedS0 = solS0[1]
            print("Computed s0 : ",computedS0)
            alpha_num = sol.subs({asymb: aval for asymb, aval in zip(a, avalue)})
            alpha_num = alpha_num.subs({s : computedS0})
            alpha_num = alpha_num.subs({tau : value_tau})
            alpha_num_eval = [i.evalf() for i in alpha_num]
            print ("Gains :",alpha_num_eval)
            alpha_sens = alpha_num_eval
            
        if user_choice == 's0' :
            
            value_s0 = user_value
            
            solTau = finaleq.subs({s : value_s0})
            solTau = sp.solve(solTau)
            
            computedTau = solTau[0]
            print("Computed tau : ",computedTau)
            alpha_num = sol.subs({asymb: aval for asymb, aval in zip(a, avalue)})
            alpha_num = alpha_num.subs({tau : computedTau})
            alpha_num = alpha_num.subs({s : value_s0})
            alpha_num_eval = [i.evalf() for i in alpha_num]
            print ("Gains :",alpha_num_eval)
            alpha_sens = alpha_num_eval
        
    with output_output_equation_computation_mid_co :
        IntProgress_mid_co.value = 6
        print ("Eq = ",polyAdm)
    
    with output_sensibilite_computation_mid_co :
        
        if user_choice == 'tau' :
            
            value_tau = user_value
        
        elif user_choice == 's0' :
            
            if type(computedTau).__name__ == "Add":
                print ("computedTau avant transformation Add: ",computedTau)
                value_tau = computedTau.as_real_imag()[0].evalf(10)
            elif type(computedTau).__name__ == "Float":
                print ("computedTau avant transformation Float: ",computedTau)
                value_tau = float(computedTau.evalf(10))
            
            value_tau = round(value_tau,5)
            print ("computedTau apres arrondissement: ",value_tau)
        
        # additional calculations for the plot
        IntProgress_mid_co.value = 7
        tau_sens,s0_sens,colormapper = compute_sensibilite(value_tau,Q,m,s,SysDerivatif,alpha,a,tau)
        IntProgress_mid_co.value = 8
        
        scaler = lambda x : [i / 255 for i in x]
        fig,ax = plt.subplots()

        for i in range(len(tau_sens)):
            xroot, yroot = getRoots(1, Q, s0_sens[i], tau_sens[i],avalue, alpha_sens, [-5, 5], [-5, 5],s,a,alpha,tau)
            ax.scatter(xroot, yroot, c=len(xroot)*[colormapper.to_rgba(float(tau_sens[i]))])
            
        IntProgress_mid_co.value = 9
        ax.grid()
        ax.set_axisbelow(True)
        fig.colorbar(colormapper)

        ax.set_xlabel(r"$Re(s)$")
        ax.set_ylabel(r"$Im(s)$")
        ax.set_title(r"Sensitivity plot, $\tau \in $"+f"[{tau_sens[0]}, {tau_sens[-1]}]")
        tau_sens.sort()
        
        IntProgress_mid_co.value = 10
        valid_statut_mid_co.value = True

    
def button_ready_to_compute_clicked_mid_co(self):
    
    valid_statut_mid_co.value = False
    IntProgress_mid_co.value = 1
    
    n_mid_co = slider_n_mid_co.value
    m_mid_co = slider_m_mid_co.value
    
    a_coeff_mid_co = []
    
    for i in range (slider_n_mid_co.value):
        a_coeff_mid_co.append(items_IntText_a_mid_co[i].value)
          
    user_choice = ToggleButtons_mid_co.value
    
    user_value = FloatText_s0_tau_mid_co.value
        
    IntProgress_mid_co.value = 2
    
    computation_2_mid_co(n_mid_co,m_mid_co,a_coeff_mid_co,user_choice,user_value)
    
    
def button_confirm_degrees_clicked_mid_co(self):
    
    for k in range (len(items_IntText_a_mid_co)):
        
        items_IntText_a_mid_co[k].value = 0
        items_IntText_a_mid_co[k].disabled= True
        items_IntText_a_mid_co[k].description = ''
    
    for i in range (slider_n_mid_co.value):

        items_IntText_a_mid_co[i].value = 1
        items_IntText_a_mid_co[i].disabled= False
        items_IntText_a_mid_co[i].description = "a"+ str(i)
    
    items_IntText_a_mid_co[slider_n_mid_co.value].value = 1
    items_IntText_a_mid_co[slider_n_mid_co.value].disabled= True
    items_IntText_a_mid_co[slider_n_mid_co.value].description = "a"+ str(slider_n_mid_co.value)
        
def button_confirm_system_coeff_clicked_mid_co(self):
    
    n_mid_co = slider_n_mid_co.value
    m_mid_co = slider_m_mid_co.value
    
    a_coeff_mid_co = []
    
    for i in range (slider_n_mid_co.value + 1):
        a_coeff_mid_co.append(items_IntText_a_mid_co[i].value)
    
    computation_1_mid_co(n_mid_co,m_mid_co,a_coeff_mid_co)
    
    
button_ready_to_compute_mid_co.on_click(button_ready_to_compute_clicked_mid_co)

button_confirm_degrees_mid_co.on_click(button_confirm_degrees_clicked_mid_co)

button_confirm_system_coeff_mid_co.on_click(button_confirm_system_coeff_clicked_mid_co)

# -----------------------------------------------------------------------------------------------------------------

def computation_CRRID(n,m,s0,d,value_tau):
    output_CRRID_root_computation.clear_output()
    output_output_equation_computation_CRRID.clear_output()
    
    with output_CRRID_root_computation :
        fig, ax = plt.subplots(figsize=(9, 5))
        ax.set_title("Spectrum distribution with equidistributed rightmost spectral values")
        ax.set_xlabel(r"Re$(s)$")
        ax.set_ylabel(r"Im$(s)$")
        ax.axhline(linewidth=2, color='black', zorder = 2)
        ax.axvline(linewidth=2, color='black', zorder = 2)
        ax.axvspan(0, 100, alpha=0.5, color='red', zorder = 2, label='Unstable system')
        ax.grid()
        line, = ax.plot([], [], 'o', color='steelblue', zorder = 3, label='Roots')
        line2, = ax.plot([], [], '*', markersize=15, color='green', zorder = 3, label='Rightmost root')
        
        xr,yr,eq = compute_CRRID_root (n,m,s0,d,value_tau)
        plt.xlim([-10,1])
        line.set_data(xr, yr)
        xmax2 = max(xr)
        line2.set_data(xmax2,0)
        ax.legend(loc='upper right')
        ax.relim()
        #ax.autoscale_view()
        fig.canvas.draw()
        IntProgress_CRRID.value = 9
        
    with output_output_equation_computation_CRRID :
        print ("Fp = ",eq)
        

def button_ready_to_compute_clicked_CRRID(self):
    
    valid_statut_CRRID.value = False
    IntProgress_CRRID.value = 1
    
    n_CRRID = slider_n_CRRID.value
    m_CRRID = slider_m_CRRID.value
    s0_CRRID = FloatText_s0_CRRID.value
    d_CRRID = slider_d_CRRID.value
    tau_CRRID = FloatText_tau_CRRID.value
    
    IntProgress_CRRID.value = 2
    
    computation_CRRID(n_CRRID,m_CRRID,s0_CRRID,d_CRRID,tau_CRRID)
    
    IntProgress_CRRID.value = 10
    valid_statut_CRRID.value = True
    
button_ready_to_compute_CRRID.on_click(button_ready_to_compute_clicked_CRRID)

# -----------------------------------------------------------------------------------------------------------------
# User Interface output
# -----------------------------------------------------------------------------------------------------------------

description_label_mid_generic = widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, rightmost root s0 and delay tau, then push the button to compute : ')

description_label_mid_co = widgets.Label('Insert degree of polynomial n and degree of delay polynomial m, then push the button to confirm : ')
description_label_2_mid_co = widgets.Label('Insert values of system coefficients, then push the button to confirm and draw the admissibility plot : ')
description_label_3_mid_co = widgets.Label('Select between rightmost root s0 or delay tau using the admissibility plot, then push the button to compute : ')
label_CRRID = widgets.VBox( [widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, rightmost root s0, step d between each value of s0 and delay tau, then push the button to compute :')],layout=widgets.Layout(width='100%',display='flex',align_items='center'))

box_input_mid_generic = widgets.HBox([slider_n_mid_generic, slider_m_mid_generic, FloatText_s0_mid_generic, FloatText_tau_mid_generic])
box_input_mid_co = widgets.HBox([slider_n_mid_co, slider_m_mid_co, button_confirm_degrees_mid_co])
box_input_2_mid_co = widgets.HBox([IntText_a_1_mid_co],layout = widgets.Layout( width='auto'))
box_input_3_mid_co = widgets.HBox([IntText_a_2_mid_co, button_confirm_system_coeff_mid_co],layout = widgets.Layout( width='auto'))
box_input_4_mid_co = widgets.HBox([ToggleButtons_mid_co,FloatText_s0_tau_mid_co],layout = widgets.Layout( width='auto'))
box_input_CRRID = widgets.HBox([slider_n_CRRID, slider_m_CRRID, slider_d_CRRID])

accordion_mid_generic = widgets.Accordion(children=[output_root_spectrum_computation_mid_generic, output_output_equation_computation_mid_generic, output_factorized_integral_equation_computation_mid_generic, output_factorization_1f1_equation_computation_mid_generic])
accordion_mid_generic.set_title(0, 'Spectrum distribution')
accordion_mid_generic.set_title(1, 'Output equation')
accordion_mid_generic.set_title(2, 'Factorized integral equation')
accordion_mid_generic.set_title(3, 'Hypergeometric factorization')

accordion_1_mid_co = widgets.Accordion(children=[output_admissibilite_computation_mid_co])
accordion_1_mid_co.set_title(0, 'Admissibility plot')

accordion_2_mid_co = widgets.Accordion(children=[output_solution_mid_co, output_output_equation_computation_mid_co, output_sensibilite_computation_mid_co])
accordion_2_mid_co.set_title(0, 'Solution')
accordion_2_mid_co.set_title(1, 'Output equation')
accordion_2_mid_co.set_title(2, 'Sensitivity plot')

accordion_CRRID = widgets.Accordion(children=[output_CRRID_root_computation, output_output_equation_computation_CRRID])
accordion_CRRID.set_title(0, 'Spectrum distribution')
accordion_CRRID.set_title(1, 'Output equation')

box_mid_generic = widgets.VBox([Text_error_mid_generic, description_label_mid_generic, box_input_mid_generic, layout_validate_mid_generic, accordion_mid_generic ])

box_mid_co = widgets.VBox([Text_error_mid_co, description_label_mid_co, box_input_mid_co, description_label_2_mid_co, box_input_2_mid_co, box_input_3_mid_co, accordion_1_mid_co, description_label_3_mid_co, box_input_4_mid_co, layout_validate_mid_co, accordion_2_mid_co ])

box_CRRID = widgets.VBox([Text_error_CRRID, label_CRRID, box_input_CRRID,layout_validate_CRRID_0, layout_validate_CRRID_1, accordion_CRRID])

tab = widgets.Tab([box_mid_generic, box_mid_co, box_CRRID])
tab.set_title(0, 'MID Generic')
tab.set_title(1, 'MID Control oriented')
tab.set_title(2, 'CRRID Generic')

file = open("images/header_p3d_online.png", "rb")
image = file.read()
header_image = widgets.Image(value=image,format='png',width=980,height=25)

dashboard = widgets.VBox([header_image,tab])
display(dashboard)